In [ ]:
# packages
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# raw data
air_raw = pd.read_csv('/Users/roshanmehta/Downloads/PSTAT/PSTAT 100/Projects/MP1/air-quality.csv')
cbsa_info = pd.read_csv('/Users/roshanmehta/Downloads/PSTAT/PSTAT 100/Projects/MP1/cbsa-info.csv')

In [ ]:
air_raw.head() # this is not tidy as all the years are in different rows and we are not able to oberve missing data

In [ ]:
cbsa_info.head(10) # we are able to see that observations were taken over different territories, and this will need to be handled

In [18]:
# merging data
data = pd.merge(air_raw, cbsa_info, how = 'left', on = 'CBSA')

# combining columns
data['Pollutant statistic'] = data[['Pollutant','Trend Statistic']].agg('-'.join, axis=1)

# dropping irrelevant columns
data.drop(columns = ['Pollutant', 'Trend Statistic', 'Number of Trends Sites'])

# reordering columns
data = data.loc[:,['CBSA','Core Based Statistical Area','Pollutant statistic', "2000", "2001", "2002", 
                   "2003", "2004", "2005", '2006', "2007", "2008", "2009", "2010", "2011", "2012", 
                   "2013", "2014", "2015", "2016", "2017", "2018", "2019"]]

data.head(3)

,CBSA,Core Based Statistical Area,Pollutant statistic,2000,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10100,"Aberdeen, SD",PM10-2nd Max,50.0,58.0,59.0,66.0,39.0,48.0,51.0,...,46.0,29.0,62.0,66.0,36.0,43.0,65.0,40.0,49.0,35.0
1,10100,"Aberdeen, SD",PM2.5-Weighted Annual Mean,8.6,8.6,7.9,8.4,8.1,9.0,8.2,...,8.7,7.1,7.5,7.3,6.2,6.2,5.4,5.8,6.6,5.9
2,10100,"Aberdeen, SD",PM2.5-98th Percentile,23.0,23.0,20.0,21.0,23.0,23.0,21.0,...,27.0,18.0,23.0,22.0,17.0,14.0,14.0,13.0,22.0,18.0


In [19]:
# tidying the data
# we first need to melt the years into a single 'year' variable and then pivot
tidy_data = data.copy()
tidy_data = tidy_data.drop('Core Based Statistical Area', axis = 1)
tidy_data = tidy_data.melt(
    id_vars =['CBSA', 'Pollutant statistic'],
    value_vars = ["2000", "2001", "2002", "2003", "2004", "2005", '2006', "2007", "2008", "2009", "2010", "2011",
                 "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"],
    var_name = 'Year',
    value_name = 'Concentration'
).pivot_table(
    index = ['CBSA', 'Year'], 
    columns = ['Pollutant statistic'], 
    values = 'Concentration').reset_index()

tidy_data.head()

Pollutant statistic,CBSA,Year,CO-2nd Max,NO2-98th Percentile,NO2-Annual Mean,O3-4th Max,PM10-2nd Max,PM2.5-98th Percentile,PM2.5-Weighted Annual Mean,Pb-Max 3-Month Average,SO2-99th Percentile
0,10100,2000,NaN,NaN,NaN,NaN,50.0,23.0,8.6,NaN,NaN
1,10100,2001,NaN,NaN,NaN,NaN,58.0,23.0,8.6,NaN,NaN
2,10100,2002,NaN,NaN,NaN,NaN,59.0,20.0,7.9,NaN,NaN
3,10100,2003,NaN,NaN,NaN,NaN,66.0,21.0,8.4,NaN,NaN
4,10100,2004,NaN,NaN,NaN,NaN,39.0,23.0,8.1,NaN,NaN


### Initial Exploratory Data Analysis

In [20]:
tidy_data.corr(method = "pearson").round(3)

Pollutant statistic,CBSA,CO-2nd Max,NO2-98th Percentile,NO2-Annual Mean,O3-4th Max,PM10-2nd Max,PM2.5-98th Percentile,PM2.5-Weighted Annual Mean,Pb-Max 3-Month Average,SO2-99th Percentile
Pollutant statistic,,,,,,,,,,
CBSA,1.000,-0.101,-0.202,-0.076,-0.018,-0.118,0.015,-0.031,0.297,0.042
CO-2nd Max,-0.101,1.000,0.566,0.609,0.329,0.345,0.283,0.218,0.029,0.314
NO2-98th Percentile,-0.202,0.566,1.000,0.871,0.573,0.392,0.462,0.563,-0.144,0.275
NO2-Annual Mean,-0.076,0.609,0.871,1.000,0.521,0.416,0.424,0.431,-0.172,0.279
O3-4th Max,-0.018,0.329,0.573,0.521,1.000,0.099,0.558,0.681,0.040,0.349
PM10-2nd Max,-0.118,0.345,0.392,0.416,0.099,1.000,0.080,-0.046,-0.251,0.020
PM2.5-98th Percentile,0.015,0.283,0.462,0.424,0.558,0.080,1.000,0.739,-0.066,0.473
PM2.5-Weighted Annual Mean,-0.031,0.218,0.563,0.431,0.681,-0.046,0.739,1.000,0.027,0.638
Pb-Max 3-Month Average,0.297,0.029,-0.144,-0.172,0.040,-0.251,-0.066,0.027,1.000,0.210


In [22]:
tidy_data.isna().sum() # there is a significant amount of missing data for several different variables

Pollutant statistic
CBSA                             0
Year                             0
CO-2nd Max                    5840
NO2-98th Percentile           5680
NO2-Annual Mean               5240
O3-4th Max                    1340
PM10-2nd Max                  4960
PM2.5-98th Percentile         2740
PM2.5-Weighted Annual Mean    2740
Pb-Max 3-Month Average        6720
SO2-99th Percentile           5240
dtype: int64

In [24]:
## PART I - using the merged, clean data
##########

# number of CBSAs included in the data
print(data.CBSA.nunique()) # there are 351 unique CBSA

# in how many states and territories do the CBSA's reside?
data_mod1 = data.copy()
data_mod1[['City','State/Territory']] = data_mod1['Core Based Statistical Area'].str.split(", ",expand=True)
data_mod1['State/Territory'].str.split('-').explode('State/Territory').nunique()
print(data_mod1['State/Territory'].nunique())
# there are 86 unique territories in the data set - the two territories are PR and DC. 


# In which years were data values recorded?
print(data.columns[3:24]) # we can see that the data was recorded for years 2000-2019.

# How many observations are recorded? / How many variables are measured?
# we will use the tidied data set
print(tidy_data.shape) # 7020 observations were recorded and 9 different variables were measured over a 20 yr. period

# Which variables are non-missing most of the time (i.e., in at least 50% of instances)?
# 7020/2 = 3510
# from this, we can see that the variables 'O3-4th Max', 'PM2.5-98th Percentile', & 'PM2.5-Weighted Annual Mean'
# are non-missing most of the time since they are over 50%.
(7020 - tidy_data.isna().sum()) / int(tidy_data.shape[0]) * 100

351
86
Index(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019'],
      dtype='object')
(7020, 11)


Pollutant statistic
CBSA                          100.000000
Year                          100.000000
CO-2nd Max                     16.809117
NO2-98th Percentile            19.088319
NO2-Annual Mean                25.356125
O3-4th Max                     80.911681
PM10-2nd Max                   29.344729
PM2.5-98th Percentile          60.968661
PM2.5-Weighted Annual Mean     60.968661
Pb-Max 3-Month Average          4.273504
SO2-99th Percentile            25.356125
dtype: float64

#### What is PM 2.5 and why is it important?
PM is particulate matter and 2.5 specifies the size of that particle (2.5 microns or less in diameter). Particulate matter is airborne and not just a single pollutant. It is made up of a mixture of many different chemical substances. For example, the combustion of gasoline, oil, diesel fuel, or wood produce create emmisions, and these make up much of the PM2.5 pollution found the air outdoors. This is important to measure and keep track of because PM2.5 can travel into our respiratory tract, reaching the lungs and even enter the blood stream, posing huge health risks. Exposure to increased levels of PM2.5 for extended periods of time is associated with increased chances of early mortality, lung cancer, and heart disease.

In [28]:
# Has PM 2.5 air pollution improved in the U.S. on the whole since 2000?

MeanP25 = tidy_data.loc[:,['Year', 'PM2.5-Weighted Annual Mean','PM2.5-98th Percentile']].groupby(
    'Year').mean().reset_index().rename(
    columns= {'PM2.5-Weighted Annual Mean': 'PM2.5 Weighted Avg. Mean', 'PM2.5-98th Percentile': 'PM2.5 98th Perc.'})

MeanP25 = MeanP25.melt(id_vars = 'Year',
    var_name = 'PM25 Statistic',
    value_name = 'P25 Value').reset_index()

alt.Chart(MeanP25).mark_line().encode(
    x = alt.X('Year:T', scale = alt.Scale(zero = False)),
    y = alt.Y('P25 Value', title = 'Overall PM 2.5', scale = alt.Scale(zero = False)),
    color = alt.Size('PM25 Statistic')
).properties(
    width = 300, 
    height = 300, 
    title=' PM2.5 Air Pollution in the U.S. from 2000-2019'
)

alt.Chart(...)

In [30]:
# Over time, has PM 2.5 pollution become more variable, less variable, 
# or about equally variable from city to city in the U.S.?

SD_P25 = tidy_data.loc[:,['Year', 'PM2.5-Weighted Annual Mean']].groupby(#,'PM2.5-98th Percentile' ]].groupby(
    'Year').std().reset_index().rename(
    columns= {'PM2.5-Weighted Annual Mean': 'PM2.5 Weighted Avg. Mean'})#, 'PM2.5-98th Percentile': 'PM2.5 98th Perc.'})

SD_P25 = SD_P25.melt(id_vars = 'Year',
    var_name = 'PM25 Statistic',
    value_name = 'P25 Value').reset_index()

alt.Chart(SD_P25).mark_line().encode(
    x = alt.X('Year:T', scale = alt.Scale(zero = False)),
    y = alt.Y('P25 Value', title = 'Standard Deviation of PM 2.5', scale = alt.Scale(zero = False)),
    color = alt.Size('PM25 Statistic')
).properties(
    width = 300, 
    height = 300,
    title='Standard Deviation of PM2.5 Weighted Avg Mean from 2000-2019'
)

alt.Chart(...)

In [33]:
# Which state has seen the greatest improvement in PM 2.5 pollution over time? Montana!

# merging data again and combining columns
x = pd.merge(air_raw, cbsa_info, how = 'left', on = 'CBSA')
x['Pollutant statistic'] = x[['Pollutant','Trend Statistic']].agg('-'.join, axis=1)

# splitting the area column
x[['City','State']] = x['Core Based Statistical Area'].str.split(", ",expand=True)
x_mod1 = x.assign(State = x['State'].str.split('-')).explode('State')

# melting the dataframe
x_mod1 = x_mod1.reset_index().melt(
    id_vars =['CBSA', 'Pollutant statistic', 'Core Based Statistical Area', 'City', 'State'],
    value_vars = ["2000", "2001", "2002", "2003", "2004", "2005", '2006', "2007", "2008", "2009", "2010", "2011",
                 "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"],
    var_name = 'Year',
    value_name = 'Concentration').pivot_table(
    index = ['CBSA', 'Year', 'City', 'State'], 
    columns = ['Pollutant statistic'], 
    values = 'Concentration').reset_index()
x_mod1 = x_mod1[['CBSA', 'Year', 'City', 'State', 'PM2.5-Weighted Annual Mean']]

x_mod1 = x_mod1.drop(columns = ["CBSA"]).groupby(['Year', 'City', 'State']).mean()
x_mod1 = x_mod1.reset_index().pivot_table(
    index = "State",
    columns = 'Year',
    values = 'PM2.5-Weighted Annual Mean')

# change in PM2.5 between 2000 and 2019
x_mod1['change'] = ((x_mod1['2019'] - x_mod1['2000']) / x_mod1['2000'])
x_mod1.sort_values(by=['change']).round(3).head()

Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,change
State,,,,,,,,,,,,,,,,,,,,,
MT,13.500,7.000,6.800,9.700,8.500,10.300,10.800,12.800,10.100,9.800,...,9.900,11.200,9.600,9.100,10.500,7.300,13.300,5.600,5.300,-0.607
VA,14.725,14.000,13.425,13.250,13.275,13.775,12.750,12.575,11.325,9.275,...,9.700,8.675,8.150,8.375,7.975,7.150,7.225,6.925,6.850,-0.535
TN,16.600,14.933,14.200,14.500,13.533,14.800,13.633,14.233,11.667,10.167,...,10.667,9.633,9.233,9.433,8.367,8.333,8.067,7.767,7.800,-0.530
AL,16.944,14.500,13.756,14.089,13.522,14.333,13.900,14.322,11.989,10.300,...,11.267,9.667,9.067,9.533,9.089,8.078,8.289,7.800,8.056,-0.525
NH,10.400,10.433,10.133,10.100,9.667,10.167,9.533,9.233,8.733,8.333,...,8.267,7.867,6.967,6.833,6.833,5.333,5.167,5.333,4.967,-0.522


In [34]:
# Which city has seen the greatest improvement? Portsmouth with a 68% decrease

x = pd.merge(air_raw, cbsa_info, how = 'left', on = 'CBSA')
x['Pollutant statistic'] = x[['Pollutant','Trend Statistic']].agg('-'.join, axis=1)

x[['City','State']] = x['Core Based Statistical Area'].str.split(", ",expand=True)
#x['State/Territory'].str.split('-', expand=True)
#x['State/Territory'].str.split('-').explode('State/Territory')
x_mod1 = x.assign(State = x['State'].str.split('-')).explode('State')

x_mod1 = x_mod1.reset_index().melt(
    id_vars =['CBSA', 'Pollutant statistic', 'Core Based Statistical Area', 'City', 'State'],
    value_vars = ["2000", "2001", "2002", "2003", "2004", "2005", '2006', "2007", "2008", "2009", "2010", "2011",
                 "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"],
    var_name = 'Year',
    value_name = 'Concentration').pivot_table(
    index = ['CBSA', 'Year', 'City', 'State'], 
    columns = ['Pollutant statistic'], 
    values = 'Concentration').reset_index()
x_mod1 = x_mod1[['CBSA', 'Year', 'City', 'State', 'PM2.5-Weighted Annual Mean']]

x_mod1 = x_mod1.drop(columns = ["CBSA"]).groupby(['Year', 'City', 'State']).mean()
x_mod1 = x_mod1.reset_index().pivot_table(
    index = "City",
    columns = 'Year',
    values = 'PM2.5-Weighted Annual Mean')

# change in PM2.5 between 2000 and 2019
x_mod1['change'] = ((x_mod1['2019'] - x_mod1['2000']) / x_mod1['2000'])
x_mod1.sort_values(by=['change']).round(3).head()

Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,change
City,,,,,,,,,,,,,,,,,,,,,
Portsmouth,21.1,20.3,16.7,14.7,12.9,16.2,14.3,14.0,12.1,10.9,...,10.1,9.8,9.0,8.2,8.5,8.3,6.9,7.1,6.7,-0.682
Reno,8.9,10.3,9.2,7.3,7.9,8.9,7.6,8.0,10.2,7.9,...,6.7,6.0,10.1,7.6,7.6,6.5,7.4,8.0,3.0,-0.663
Kingsport-Bristol-Bristol,16.6,15.1,14.1,13.8,13.8,14.3,13.5,13.9,10.6,9.2,...,9.8,8.8,8.5,8.6,7.4,8.0,7.2,6.7,6.4,-0.614
Butte-Silver Bow,13.5,7.0,6.8,9.7,8.5,10.3,10.8,12.8,10.1,9.8,...,9.9,11.2,9.6,9.1,10.5,7.3,13.3,5.6,5.3,-0.607
Asheville,15.4,13.5,13.8,12.6,12.3,13.1,12.4,12.2,9.0,8.4,...,9.2,8.6,8.1,7.9,7.0,8.5,6.7,6.0,6.1,-0.604


In [35]:
# Choosing a specific location & checking if that place was in compliance with 
# EPA primary standards as of the most recent measurement.

data_mod1.loc[data_mod1['City'] == 'Anchorage']

# 2.4 is less than 9 ppm | 7.4 is less than 12 µg/m3 | 35 at the max standard level of 35.0

,CBSA,Core Based Statistical Area,Pollutant statistic,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,City,State/Territory
33,11260,"Anchorage, AK",CO-2nd Max,5.4,5.7,4.7,5.7,6.4,4.8,4.3,...,4.3,3.1,2.5,2.8,3.0,3.5,2.7,2.4,Anchorage,AK
34,11260,"Anchorage, AK",PM2.5-Weighted Annual Mean,5.8,6.2,5.8,6.6,6.9,6.7,6.9,...,6.2,5.6,7.0,6.5,6.1,5.6,5.0,7.4,Anchorage,AK
35,11260,"Anchorage, AK",PM2.5-98th Percentile,20.0,25.0,25.0,23.0,30.0,22.0,33.0,...,31.0,22.0,29.0,28.0,23.0,28.0,18.0,35.0,Anchorage,AK
